I made lists of barcodes using the extractBCs method in another python notebook in this folder

Here I'm reloading the lists and generating a list of counts for each unique barcode in a single dataframe

I'm including the PacBio data, both raw and filtered



Still need:

Use Bokeh for all graphs

Also plot

- Averages of barcodes of a given mutant
    - get aggregate value and plot
    - use error between barcodes

Heatmap

- number of barcodes
- error


In [1]:
import numpy as np
import itertools
import math
import pandas as pd
import time
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
import matplotlib
import matplotlib.pyplot as plt
from ast import literal_eval
import csv
import venn
from numpy import cov
from scipy.stats import spearmanr, pearsonr
from numpy import cov
from scipy.stats import spearmanr

experimentName = 'NP_11_27_selection_first'
listOfFileNames = ['NP_11_27_23', 'NP_11_27_24', 'NP_11_27_25']


In [2]:
# Running this cell will stop the jupyter notebook from opening a new tab if that starts happening

import bokeh.io
from bokeh.layouts import gridplot
from bokeh.io import show, export_svgs, export_png
from bokeh.transform import transform
from bokeh.plotting import ColumnDataSource, output_file, figure, show, save, output_notebook, reset_output
from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, LogColorMapper, PrintfTickFormatter, Whisker, Title, NumeralTickFormatter, Label, LabelSet, Text, HoverTool
reset_output()
output_notebook()

Loading BokehJS ...

In [3]:
combinedDF = pd.read_csv(experimentName + '.csv')


/Users/noamprywes/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2,7,8,9,10,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
# Heatmap showing each individual mutation read frequency
# This will only give the barcode heatmap for now
# This is messed up, figure out what's going on with dataframe
positionAAFreq = combinedDF.groupby(['AAPosition', 'MutAA'])['WT_norm_enrich_avg'].mean()
dfPositionAAFreq = positionAAFreq.to_frame(name = 'WT_norm_enrich_avg')
pivotBCs = combinedDF.pivot_table(index = 'MutAA', columns = 'AAPosition', values = 'WT_norm_enrich_avg')

# Convert pivot table to regular dataframe
pivotBCsReg = pd.DataFrame(pivotBCs.to_records()).set_index('MutAA')

fig, (ax1) = plt.subplots(1, tight_layout=True, figsize=(10,5), dpi = 100)
pcmBCs = ax1.pcolor(pivotBCsReg, cmap = 'terrain', norm=matplotlib.colors.LogNorm())

ax1.set_title('Average enrichment per mutant', size = 15)
ax1.set_xlabel('Amino acid position in rbcL', size = 15)
ax1.set_ylabel('Amino acid', size = 15)
# This sets the ticks in the middle
ax1.set_yticks(np.arange(0.5, 20.5))
ax1.set_yticklabels(pivotBCs.index)


plt.xticks(np.arange(0.5, len(pivotBCsReg.columns), 1), pivotBCsReg.columns)
plt.xlim(180,200)

fig.colorbar(pcmBCs, ax = ax1)

# xAxisVals = pivotBCsReg.columns
# visibleTicks = [::20]


plt.setp(ax1.get_xticklabels()[::2], visible=False)


plt.savefig('NP-11-27-heatmap.pdf')

plt.show()

ValueError: name already used as a name or title

In [30]:
# Median enrichment value by position

plt.rcParams["figure.figsize"] = (15,5)

plt.scatter(pivotBCsReg.median().index, pivotBCsReg.median())

plt.savefig('NP-11-27-medianEnrichmentByPos.pdf')


NameError: name 'pivotBCsReg' is not defined

In [40]:
from bokeh.layouts import row
from bokeh.plotting import figure, output_file, show

factors = ["a", "b", "c", "d", "e", "f", "g", "h"]
x =  [50, 40, 65, 10, 25, 37, 80, 60]

dot = figure(title="Categorical Dot Plot", tools="", toolbar_location=None,
            y_range=factors, x_range=[0,100])

dot.segment(0, factors, x, factors, line_width=2, line_color="green", )
dot.circle(x, factors, size=15, fill_color="orange", line_color="green", line_width=3, )

factors = ["foo 123", "bar:0.2", "baz-10"]
x = ["foo 123", "foo 123", "foo 123", "bar:0.2", "bar:0.2", "bar:0.2", "baz-10",  "baz-10",  "baz-10"]
y = ["foo 123", "bar:0.2", "baz-10",  "foo 123", "bar:0.2", "baz-10",  "foo 123", "bar:0.2", "baz-10"]
colors = [
    "#0B486B", "#79BD9A", "#CFF09E",
    "#79BD9A", "#0B486B", "#79BD9A",
    "#CFF09E", "#79BD9A", "#0B486B"
]

hm = figure(title="Categorical Heatmap", tools="hover", toolbar_location=None,
            x_range=factors, y_range=factors)

hm.rect(x, y, color=colors, width=1, height=1)


show(hm)  # open a browser

In [41]:
# Bokeh heatmap

df = pd.DataFrame(dfPositionAAFreq).reset_index()

df = df.astype({'AAPosition': 'str'})

source = ColumnDataSource(df)

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
colors = 'Inferno256'
mapper = LogColorMapper(palette=colors, low=df.avgEnrich.min(), high=df.avgEnrich.max())

TOOLTIPS = [
    ("Mutant", "@MutAA"),
    ("Position", '@AAPosition'),
    ("Average enrichment", '@avgEnrich'),
]

TOOLS="crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save"


p = figure(
#            sizing_mode="scale_width", \
           plot_width=1000, \
#            plot_height=300, \
           title="Average enrichment heatmap", \
           x_range = list(df['AAPosition'].unique()), \
           y_range=list(df['MutAA'].unique()), \
           toolbar_location='below', \
           tools = TOOLS, \
#            tooltips=TOOLTIPS, \
           x_axis_location="above")

p.rect(
       x="AAPosition", \
       y="MutAA", \
       width=1, \
       height=1, \
       source=source, \
       line_color=None, \
       fill_color=transform('avgEnrich', mapper))

color_bar = ColorBar(color_mapper=mapper,
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%4.1e"))

p.add_layout(color_bar, 'left')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "7px"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = 1.0


hover.tooltips = TOOLTIPS
p.tools.append(hover)

show(p)

from bokeh.io import export_svgs
p.output_backend = "svg"
export_svgs(p, filename="bokehHeatmap.svg")


['bokehHeatmap.svg']